# Worksheet 12: A/B Testing and principled peeking

## Learning Objectives 

After completing this week's worksheet and tutorial work, you will be able to:

1. Explain sequential testing and principled peeking and how it can be used for early stopping of an experiment (e.g., A/B testing).
2. Write a computer script to perform A/B testing optimization with and without using principled peeking.
3. Discuss the tradeoff between stopping earlier and certainty of significance, and thereal world implications (e.g., what does the FDA require for early stopping of clinical trials versus Facebook ads optimization?).
4. List other questions related to A/B testing optimization that may be relevant in a real data application (e.g., what features cause a Facebook ad to perform best?)

## Loading packages

In [ ]:
# Run this cell before continuing.
library(tidyverse)
library(infer)
library(broom)
library(gsDesign)

source("tests_worksheet_12.R")

# Part I

## 1. A/B Testing Optimization

**A/B testing** refers to an experiment in which users are randomly assigned to one of two variations of a product or service: control (A) and variation (B) to see if variation B should be used for improvement.

> A/B testing became very popular for updating and improving websites. However, it can also be used to monitor and update products and/or services in many other contexts.

A key factor in A/B testing is deciding which statistical tool will be used to analyze the data collected. 
For example, a classical 𝑡-test can be used to compare differences in population means. In this worksheet, we will review some fundamental concepts of hypothesis testing in the context of A/B testing problems.

-------------------

*For the questions in this section, suppose that you are designing an A/B test to determine whether a new website used to collect donations for a political campaign increases donation sizes.*

**Question 1.0**
<br>{points: 1}

Before analyzing the collected data, it is essential to decide the significance level of the test (i.e., Type I error rate). This quantity is interpreted as:

**A.** the probability of finding a significant difference between the donation sizes collected from two variations of the website when the new website indeed attracts, on average, larger donations.

**B.** the probability of *not* finding a significant difference between the donation sizes collected from two variations of the website when the new website indeed attracts, on average, larger donations.

**C.** the probability of finding a significant difference between the donation sizes collected from two variations of the website when the mean of the size of the donations of both websites are equal.

**D.** the probability that the new website indeed attracts larger donations.

*Assign your answer to an object called `answer1.0`. Your answer should be one of `"A"`, `"B"`, `"C"`, or `"D"` surrounded by quotes.*

In [ ]:
# answer1.0 <- 

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_1.0()

**Question 1.1**
<br>{points: 1}

The designers of the experiment also need to decide how large the experiment will be since there are large costs (including opportunity costs) related to the experiment. 

Thus, for the statistical test planned, they decide to conduct a **power analysis** to:

**A.** estimate the minimum sample size required, given a desired significance level, expected difference in mean donations, and statistical power.

**B.** maximize the probability of finding a significant difference between the donation sizes collected from two variations of the website.

**C.** minimize the probability of not finding a significant difference between the donation sizes collected from two variations of the website.


*Assign your answer to an object called `answer1.1`. Your answer should be one of `"A"`, `"B"`, or `"C"` surrounded by quotes.*

In [ ]:
# answer1.1 <- 

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_1.1()

**Question 1.2**
<br>{points: 1}

Consider the following scenario:

After deciding on the sample size required and *randomly* assigning visitors to each variation of the website, the company will start analyzing the size of the donations made by visitors. 

Assuming the planned sample size is large enough, the analysts will conduct CLT-based statistical inference, i.e., compute $p$-values and confidence intervals based on the CLT results.

Considering the opportunity costs involved in this experiment, the analysts will monitor the size of the donations closely and stop the experiment earlier if they find (using a standard two-sample $t$-test) that the new website attracts higher donations. 

However, computing non-adjusted $p$-values before collecting all the data in the experiment and stopping the experiment based on preliminary findings can result in a false discovery, which can lead to an unnecessary change to a new website variation and an increase in expenditure due to the costs associated with such a change.

Is the reasoning above *true* (correct) or *false* (incorrect)?

*Assign your answer to an object called answer1.2 Your answer should be either "true" or "false", surrounded by quotes.*

In [ ]:
# answer1.2 <- 

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_1.2()

## 2. Early Stopping in A/B Testing

In this section you will analyze data from an A/A experiment to examine the problem of <u>early stopping</u>:

> *Can we stop or re-design the experiment earlier if we have supporting evidence to do so?*

Note that in A/A testing, we know that $H_0$ is true (i.e., there is no effect); that's the whole purpose of A/A testing. For example, we can think of a scenario where both groups are exposed to the same website. In A/A testing, we know that claiming a significant result is a false discovery
  
Although this procedure seems artificial, it is a widely used technique to test experiments and platforms because it allows you to estimate the error rates associated with your experiment by repeating it multiple times.

![img](img/aa-Obama.png)
<font color=grey>Figure by [R. Lourenzutti](https://lourenzutti.github.io) </font>

Each simulated dataset will be analyzed according to the following steps:

1. run a balanced experiment with a *pre-set* sample size of $n$ visitors per variation 

2. sequentially collect the data in batches of visitors per group

3. sequentially analyze the data using two-sample $t$-tests

4. sequentially compute and monitor* the $p$-values (non-adjusted) 

5. stop the experiment once a significant result is found 

#### Simulation function

We have prepared a function for you that: 

- generates two samples, each of size `n`, from two (known) Normal distributions, the control and the variation. Note that in real data analysis, we collect data from an *unknown* distribution (i.e., not necessarily Normal). 

- analyzes the data in an incremental way by `sample_increase_step` until all `n` samples in each treatment group are analyzed. For example, we compare donations by batches of visitors of each website variation.

- returns the $t$-statistic and $p$-value (computed by a two-sample $t$-test) for every batch of collected data.

For example, if `sample_increase_step` is 20, and `n=500`, the function will:
1. draw the first 20 experimental units from each group;

2. perform the two-sample $t$-test and return the associated $t$-statistic and $p$-value;

3. draw 20 more experimental units for each group 

4. perform the two-sample $t$-test (now based on 40 experimental units per group) and return the associated $t$-statistic and $p$-value  

5. draw another 20 experimental units for each group 

6. perform the two-sample $t$-test (now based on 60 experimental units per group) and return the associated $t$-statistic and $p$-value 
$$
\vdots\hspace{18cm}
$$
and so on, until the total sample size in each group is 500 (as originally planned).

The function returns a tibble that has three columns:

- `inc_sample_size`: the sample size of the set of data analyzed 
- `statistic`: $t$-statistic calculated by the `t.test()` function
- `p_value`: $p$-value calculated by the `t.test()` function

<br>
<br>

_<u>Note: You don't need to understand the code for this function; it's enough to understand what the function is doing (as explained above) and the function's arguments as detailed in the comments of the code block below.</u>_

In [ ]:
# Two-sample t-test with tracking sequential statistic and p-values 
# by incremental sample sizes until getting to n in each group.

# @param n (numeric): Initially planned sample size for each group (for simplicity,
#                     n needs to be a multiple of sample_increase_step).
# @param d_0 (numeric): effect size.
# @param mean_current (numeric): Population mean for control variation.
# @param sd_current (numeric): Population standard deviation for current variation.
# @param sd_new (numeric): Population standard deviation for new variation.
# @param sample_increase_step (numeric): Sample size increment.

# @return p.value.df: A tibble that has 3 columns:
# inc_sample_size, statistic, and p_value 

incremental_t_test <- function(n, d_0, mean_current, sd_current, sd_new, sample_increase_step) {
  sample_current <- rnorm(n, mean = mean_current, sd = sd_current)
  sample_new <- rnorm(n, mean = mean_current + d_0, sd = sd_new)

  p.value.df <- tibble(
    inc_sample_size = rep(0, n / sample_increase_step),
    statistic = rep(0, n / sample_increase_step),
    p_value = rep(0, n / sample_increase_step)
  )

  current_sample_size <- sample_increase_step
  
  for (i in 1:nrow(p.value.df))
  {
    t_test_results <- t.test(sample_new[1:current_sample_size], sample_current[1:current_sample_size],
      var.equal = TRUE,
      alternative = "greater"                      
    )
    p.value.df[i, "statistic"] <- as_tibble(t_test_results$statistic)
    p.value.df[i, "p_value"] <- as_tibble(t_test_results$p.value)
    p.value.df[i, "inc_sample_size"] <- current_sample_size
    current_sample_size <- current_sample_size + sample_increase_step
  }

  return(p.value.df)
}

$$ $$

_For the questions in this section, suppose that before running an A/B test to compare the donation sizes from 2 variations of websites in a political campaign, you design an A/A testing to examine the early stopping problem._

**Question 2.0**
<br>{points: 1}

In a simulation study, we know the true population distributions! The function given to you to simulate the data assumes:

**A.** the sample distributions are $\mathcal{N}(0,1)$

**B.** the population distribution of the donation sizes of visitors of the current website is $\mathcal{N}(\mu_0,\sigma_0^2)$, where $\mu_0$ = mean_current and $\sigma_0$ = sd_current

**C.** the sample distribution of the donation sizes of visitors of the current website is $\mathcal{N}(\mu_0,\sigma_0^2)$, where $\mu_0$ = mean_current and $\sigma_0$ = sd_current

*Assign your answer to an object called `answer2.0`. Your answer should be one of `"A"`, `"B"`, or `"C"` surrounded by quotes.*

In [ ]:
# answer2.0 <- 

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_2.0()

**Question 2.1**
<br>{points: 1}

Simulate data for an A/A testing from populations with the same expected size of the donations (e.g., assuming all visitors are exposed to the same website). 

Suppose that the compaign organizers want to analyze the data in batches of 50 visitors per group until a total of $n = 1000$ visitors have watched each website.

Use the `incremental_t_test` function to conduct the company's experiment. 

*Save the result in an object called `answer2.1`. Your answer should be a tibble with three columns: `inc_sample_size`, `statistic` and `p_value`.*

In [ ]:
set.seed(301) # do not change this.

#answer2.1 <- 
#    incremental_t_test(n = ..., d_0 = ..., sample_increase_step = ..., mean_current = 200, sd_current = 50, sd_new = 50)

# your code here
fail() # No Answer - remove if you provide an answer

answer2.1

In [ ]:
test_2.1()

**Question 2.2**
<br>{points: 1}

Using the data stored in `answer2.1`, plot the $p$-value sequence as a **line** with the incremental sample size on the $x$-axis and $p$-value on the $y$-axis. Add a dashed horizontal red line that indicates a threshold of the significance level $\alpha = 0.05$. The `ggplot()` object's name will be `sequential_pvalue`.

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 9) # Adjust these numbers so the plot looks good in your desktop.

# sequential_pvalue <- 
#   answer2.1 %>%
#   ggplot(aes(x = ..., y = ...)) +
#   geom_point(size = 3) +  
#   geom_line() +
#   geom_hline(
#     yintercept = ...,
#     colour = "red",
#     linetype = "twodash"
#   ) +  
#   ggtitle("Evolution of p-values in Experiment 1") +
#   ylab("p-value") +
#   xlab("Sample Size") +
#   coord_cartesian(ylim = c(0, 1)) +
#   scale_y_continuous(breaks = seq(0, 1, by = 0.05)) +
#   theme(
#     text = element_text(size = 18),
#     plot.title = element_text(face = "bold"),
#     axis.title = element_text(face = "bold")) 


# your code here
fail() # No Answer - remove if you provide an answer

sequential_pvalue

In [ ]:
test_2.2()

**Question 2.3**
<br>{points: 1}

Suppose the campaign organizers want to implement an early stopping (before reaching the maximum sample size of `n = 1000` visitors per website) to save time and resources allocated for the experiment. They have decided to stop the experiment as soon as they find a significant result. 

Given the results in the previous question and a significance level $\alpha = 0.05$, the compaign organizers would stop the experiment 

**A.** once they finish collecting and analyzing all the data

**B.** after 100 visitors have entered each website since the $p$-value is below the specified significance level

**C.** after 150 visitors have entered each website since results are getting worse after that point.

*Assign your answer to an object called `answer2.3`. Your answer should be one of `"A"`, `"B"`, or `"C"` surrounded by quotes.*

In [ ]:
# answer2.3 <- 

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_2.3()

**Question 2.4**
<br>{points: 1}

Since this is an **A/A testing** design, what error, if any, are the campaign organizers making by stopping the experiment, as noted in the previous question?

**A.** No error.

**B.** Type I Error.

**C.** Type II Error.

*Assign your answer to an object called `answer2.4`. Your answer should be one of `"A"`, `"B"`, or `"C"` surrounded by quotes.*

In [ ]:
# answer2.4 <- 

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_2.4()

#### Type I Error Rate

In the experiment analyzed in the previous questions, you noticed that the campaign organizers made a mistake in declaring one variation more effective than the other based on data from 100 visitors in each group. However, the hypothesis test was designed with a $5\%$ probability of falsely rejecting $H_0$ and considering the new website more effective, even when it was *identical* to the alternative variation. 

> If you repeat the experiment *many* times, do you expect to *always* make a mistake? 



**Question 2.5.1**
<br>{points: 1}

If you repeat this A/A testing *many* times, at a significance level of $5\%$, how often do you expect to find differences between the two groups?  

_Assign your answer to an object called `answer2.5.1`._

In [ ]:
# answer2.5.1 <- ...

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_2.5.1()

<br>
<br>

The probability of Type I Error was specified to be $\alpha$. However, $\alpha$ is the Type I Error probability for **one** test. In this case, the campaign monitors the data as they come and tests the hypothesis each time more data is available. Therefore, the campaign is actually conducting multiple hypotheses testing. 

Let us now investigate the impact of the campaign's online p-value monitoring. 

**Question 2.5.2**
<br>{points: 1}

In this question

- perform the **A/A testing** experiment 100 times 

- count how many times you would wrongly reject $H_0$ with the previous strategy, and

- compare it with the expected number of rejections given the significance level $\alpha = 0.05$

We wrote a code to perform the first step in the cell below. 

Now, you need to work on the rest!

Your answer will be a tibble with two columns: `n_rejections` and `expected_n_rejections`.

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
set.seed(120)

### Run this before continuing
multiple_times_sequential_tests <- 
    tibble(experiment = 1:100) %>% 
    mutate(seq_test = map(.x = experiment, 
                          .f = function(x) incremental_t_test(n = 1000, d_0 = 0, sample_increase_step = 50, 
                              mean_current = 200, sd_current = 50, sd_new = 50)))


In [ ]:
#answer2.5 <- multiple_times_sequential_tests %>% 
#    mutate(reject = map_dbl(.x = seq_test, .f = function(x) sum(x$p_value< ...) > 0)) %>% 
#    summarise(n_rejections = ...(reject),
#              expected_n_rejections = ...)

# your code here
fail() # No Answer - remove if you provide an answer
                            
answer2.5

In [ ]:
test_2.5.2()

**Question 2.6**
<br>{points: 1}

Select the right option to complete the sentence below:

> *With the strategy used by the company, the probability of Type I error is approximately ... the specified one.* 

**A.** equal to

**B.** 3 times lower than

**C.** 5 times lower than

**D.** 3 times higher than

**E.** 5 times higher than

*Assign your answer to an object called `answer2.6`. Your answer should be one of `"A"`, `"B"`, `"C"`, `"D"`,  or `"E"` surrounded by quotes.*

In [ ]:
#answer2.6 <- ""

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_2.6()

<br>
<br>

**Note**: not only the type I error rate is affected by this problem, but also the estimates themselves! If we analyze samples until the means of both groups are significantly far apart, we would also overestimate the effect size (difference between means).

<font color=blue> In *classical hypothesis testing*, monitoring results in a dashboard and *stopping experiments earlier* than planned will increase the probability of *incorrectly* rejecting the null hypothesis (i.e., when there is no real effect). </font>

## Summary and conclusions:

- One may be (understandably) tempted to peek at the results of A/B tests as data are being collected.

- Stopping an experiment and rejecting $H_0$ as soon as the $p$-value is below the specified significance level can drastically inflate the type I error rate

- Controlling the risk of wrongly rejecting the null hypothesis is not an easy task in A/B testing if peeking and early stops are allowed

# Part II

## 3. Sequential testing 

**Sequential tests** are decision rules that allow users to test data sequentially as data come in. 

- Users *are* monitoring results as they collect and analyze data and are making decisions accordingly.

- Users need to adaptively determine the sample size of the experiments since there are large opportunity costs associated with longer experiments. 

- When done correctly, stopping an experiment earlier (or re-designing it) can be beneficial in many contexts. 

In this second part we will examine how to address the problem of *multiple testing* when sequential tests are performed in an A/B testing experiment.

### 3.1 Bonferroni's method

The Bonferroni's method has been proposed to control the overall Type I error rate when multiple tests are performed. It can be thought as: 

- an adjustment of the $p$-values, multiplying them by the number of comparisons, and keeping the significance level at a desired threshold, or 

- an adjustment of the significance threshold $\alpha$, dividing it by the number of comparisons, or

- an adjustment of the critical value, computed with a sampling distribution, corresponding to the adjusted significance threshold

<br>
<br>

<u>In this section, you are going to investigate if a Bonferroni correction controls the type I error rate in A/B testing using data from the **A/A testing** of the political campaign generated in previous questions.</u>

**Question 3.1.0**
<br>{points: 1}

Since the campaign organizers have decided to monitor the data every 50 visitors per website, up to 1000, they will perform 20 sequential tests. 

Suppose that after each interim analysis, they will use a Bonferroni correction to control the type I error rate at $5\%$. Thus, using a classical two-sample $t$ test, they we will **reject $H_0$** if the raw $p$-value is: 

**A.** smaller than 0.05

**B.** smaller than 0.0025 (adjusted threshold)

**C.** greater than a 0.0025 (adjusted threshold)

**D.** greater than 0.05 when multiplied by 4


*Assign your answer to an object called `answer3.1.0`. Your answer should be one of `"A"`, `"B"`, `"C"`, or `"D"`, surrounded by quotes.*

In [ ]:
# answer3.1.0 <- 

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_3.1.0()

**Question 3.1.1**
<br>{points: 1}

Continuing with the problem stated in **Question 3.1.0**, the campaign organizers can also **reject $H_0$** if the observed $t$-statistic is:

**A.** greater than `qt(1 - 0.05,1998) = 1.65` 

**B.** greater than `qt(1 - 0.025,1998) = 1.96`

**C.** greater than `qt(1 - 0.0025,1998) = 2.81` 

**D.** greater than 0.05


*Assign your answer to an object called `answer3.1.1`. Your answer should be one of `"A"`, `"B"`, `"C"`, or `"D"`, surrounded by quotes.*

In [ ]:
# answer3.1.1 <- 

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_3.1.1()

**Question 3.1.2**
<br>{points: 1}

In *Question 2.1*, you performed 20 interim analyses of simulated data for an A/A design. Modify the code of *Question 2.5.2* to implement a Bonferroni correction as specified in *Question 3.1.0* in 100 experiments.

Then, compare the estimated type I error rate when a Bonferroni correction is used with the expected type I error rate value.

*Assign your answer to an object called `answer3.1.2`. Your answer should be a tibble with two columns: `n_rejections_Bonf` and `expected_n_rejections`.*

In [ ]:
#answer3.1.2 <- ...

# your code here
fail() # No Answer - remove if you provide an answer
                            
answer3.1.2 

In [ ]:
test_3.1.2()

<br>
<br>
Using a Bonferroni correction, the data can be sequentially analyzed and the experiment can be stopped earlier while controlling the <u>type I error rate</u>. For the simulated experiments in the example above, the type I error rate was approximately 2%, which is now below the planned 5% value. 

The Bonferroni's correction in sequential analysis is very conservative and can affect the power of the test (i.e., we are more vulnerable to the <u>type II error</u>)!

### 3.2 Pocock boundaries

As we recalled in **Question 3.1.1**, the Bonferroni correction can be implemented by adjusting the critical value to `qt(1 - 0.0025, 1998) = 2.81`. In this section we will examine the **Pocock method** to compute alternative critical values to evaluate interim analyses in sequential A/B testing.

Similarly to Bonferroni's method, the **Pocock method** computes a *common* critical value for all interim analyses. However, the Pocock's boundary is not an adjustment of the quantile of a $t$-distribution.

We can easily get the critical values for this design using `gsDesign::gsDesign()`.

**Note 1**: `gsDesign()` outputs a full sequential design, not just the critical values to control a desired type I error rate!! You can read more about this package [here](https://keaven.github.io/gsDesign/reference/gsDesign.html)

**Note 2**: a caveat about this package is that two-sample tests are based on $z$-statistics, i.e., a case for which we assume that samples are drawn from Normal distributions with known SD. While this is usually an unrealistic assumption and in practice we use a $t$-test to compare means of two populations, results are nearly equivalent to a $z$-test. More can be read [here](https://keaven.github.io/gsDesign/articles/nNormal.html)  

In the following exercises we will examine if the critical values of the Pocock design can be used to control the type I error rate. 

Let's start computing a Pocock design. Save the output in the object `design_pocock`. Extract the Pocock's critical values for each interim analyses and save them in an object called `crit_pocock`.

In [ ]:
# Run this cell to get a Pocock design!

design_pocock <- gsDesign(k = 20, #number of interim analysis planned
                          test.type = 1, # for one-sided tests
                          delta = 0, # default effect size
                          alpha = 0.05, #type I error rate
                          beta = 0.2, # type II error rate
                          sfu = 'Pocock')
                          
crit_pocock <- design_pocock$upper$bound
crit_pocock

**Question 3.2.0**
<br>{points: 1}

As we know, when performing a hypothesis test, we can either compare the $p$-value to a pre-specified significance level $\alpha$ *or* we can compare the observered statistic to a critical value. 

Based on the adjusted critical values computed with Bonferroni's method (Question 3.1.1) and Pocock's method (cell above), the Pocock method is more conservative than the Bonferroni correction. **True or False?**

*Assign your answer to an object called answer3.2.0. Your answer should be either "true" or "false", surrounded by quotes.*

In [ ]:
#answer3.2.0 <- 

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_3.2.0()

**Question 3.2.1**
<br>{points: 1}

Using the data stored in `answer2.1`, plot the sequence of observed statistics for each interim analysis as a **line** with the incremental sample size on the $x$-axis and the value of the observed statistic on the $y$-axis. 

Add 3 dashed horizontal lines that indicate the following 3 boundaries (critical values): 

- a red line for the Pocock's critical values

- a blue line for the Bonferroni's critical values

- a black line for the unadjusted critical values

The `ggplot()` object's name will be `sequential_stat`.

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 9) # Adjust these numbers so the plot looks good in your desktop.

#crit_unadj <- qt(1 - ..., ...)
#crit_bonferroni <- ...(1 - ..., ...)

#sequential_stat <- 
#  answer2.1 %>%
#  ggplot() +
#  geom_line(aes(x = inc_sample_size, y = statistic)) +
#  geom_point(aes(x = ..., y = ...)) +
#  geom_hline(yintercept = ..., colour = "red", linetype = "twodash") +
#  geom_point(aes(x = inc_sample_size, y = ...), colour = "red") +
#  geom_text(x=850, y=crit_pocock + 0.15, size=6, label="Pocock",colour = "red") +
#  geom_hline(yintercept = ..., colour = "blue", linetype = "twodash") +
#  geom_point(aes(x = inc_sample_size, y = rep(crit_bonferroni, 20)), colour = "blue") +
#  geom_text(x=850, y=crit_bonferroni + 0.15, size=6, label="Bonferroni",colour = "blue") +
#  geom_hline(yintercept = ..., linetype = "twodash") +
#  geom_point(aes(x = inc_sample_size, y = rep(..., 20))) +
#  geom_text(x=850, y=crit_unadj + 0.15, size=6, label="Unadjusted") +
#  theme(
#    text = element_text(size = 18),
#    plot.title = element_text(face = "bold"),
#    axis.title = element_text(face = "bold")
#  ) +
#  ggtitle("Critical values in Sequential Designs") +
#  ylab("Statistic") +
#  xlab("Sample Size") +
#  coord_cartesian(ylim = c(-1, 3)) +
#  scale_y_continuous(breaks = seq(-1, 3, by = 0.5))

# your code here
fail() # No Answer - remove if you provide an answer

sequential_stat

In [ ]:
test_3.2.1()

**Question 3.2.2**
<br>{points: 1}

The compaign organizers have decided to monitor the data every 50 visitors per website and stop the experiment earlier if there's evidence of a difference between the group means. According to the data plotted **Question 3.2.1**, which of the following statement is correct?? 

**A.** The compaign organizers would never stop the experiment, regardless of the boundary used

**B.** The compaign organizers would erroneously stop the experiment after the analysis of the second test, regardless of the boundary used

**C.** The compaign organizers would erroneously stop the experiment after the analysis of the second test, only if they correct the critical values using a Bonferroni's method to control the type I error rate

**D.** The compaign organizers would erroneously stop the experiment after the analysis of the second test only if they use undadjusted $t$ critical values 

*Assign your answer to an object called `answer3.2.2`. Your answer should be one of `"A"`, `"B"`, `"C"`, or `"D"` surrounded by quotes.*

In [ ]:
# answer3.2.2 <- 

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_3.2.2()

**Question 3.2.3**
<br>{points: 1}

In *Question 2.1* you performed 20 interim analyses of simulated data for an A/A design. Modify the code of *Question 2.5* to implement a sequential analyses using Pocock's boundary to control the type I error in 100 experiments.

Then compare the estimated type I error rate when the Pocock's method is used with the expected type I error rate value.

*Assign your answer to an object called `answer3.2.3`. Your answer should be a tibble with two columns: `n_rejections_Pocock` and `expected_n_rejections`.*

In [ ]:
#answer3.2.3 <- ...

# your code here
fail() # No Answer - remove if you provide an answer
                            
answer3.2.3 

In [ ]:
test_3.2.3()

#### <font color=blue> Using the Pocock's method, the data can be sequentially analyzed and the experiment can be stopped earlier while controlling the type I error rate. </font>
- For the simulated experiments in the example above, the type I error rate was 7%, which close to the planned 5% value

- As expected, this method is less conservative than the Bonferroni's correction

In Tutorial 2, you will implement another sequential test method available in `gsDesign` package, called the **O’Brien-Fleming method**, which has conservative critical values for earlier interim analysis and less conservative values (closer to the unadjusted critical values) as more data are collected. In other words, bounds are not uniform. 

There are many other methods to implement principled peeking strategies in A/B testing. 
- A very popular and flexible method, implemented by [Optimizely](https://www.optimizely.com), computes a mixture sequential probability ratio test (mSPRT) tests and *always valid* $p$-values. The metholology and implementation are beyond the scope of this course but here's a nice [video](https://www.youtube.com/watch?v=AJX4W3MwKzU) that explains its key points without too many technical details.

## 4. Summary and key concepts learned

1. A/B testing refers to an experiment in which users are randomly assigned to one of two variations of a product or service: control (A) and variation (B) to see if variation B should be used for improvement.


2. The statistic used to test a hypothesis, the sample size calculation, the type I error rate specification, and the desired power are all important and interconnected pieces of the experimental design! 


3. In classical hypothesis testing theory, the sample size must be fixed in advance when the experiment is designed!!


4. Modern platforms allow users to continuously monitor their tests' p-values and confidence intervals as data are collected (peeking) to dynamically re-adjust their experiments. 


5. In particular, users would like to stop their experiments earlier depending on the results of interim analyses


6. Naively stopping experiments earlier than planned will increase the probability of *incorrectly* rejecting the null hypothesis (i.e., when there is no real effect). Stops must be part of the experimental design and appropriate testing methods must be used!


7. Sequential testing triggers a multiple comparison problem. If you make many comparisons but don't correct them, error rates are inflated!! A particular characteristic of this setting is that tests are nested and not independent.


8. A possible way to control the type I error rated is to use a Bonferroni adjustment of the $p$-values (or equivalently, the significance level or critical values). As with other multiple comparison problems, Bonferroni's correction in sequential analysis is very conservative and can affect the power of the test!!


9. The Pocock's method offers a less conservative way of controlling the type I error rate in sequential testing with early stops.


10. *Principled* peeking is ok and even beneficial in A/B testing.

> The experimental design is a very important piece of any statistical analysis! 